# EXAMEN

In [28]:
def secant_method(f, x0, x1, tol=1e-6, max_iter=25):
    """
    Secant method for finding the root of a function.

    # Parameters
    * ``f``: The function for which to find the root.
    * ``x0``, x1: Initial guesses for the root.
    * ``tol``: Tolerance for convergence (default: 1e-6).
    * ``max_iter``: Maximum number of iterations (default: 25).

    # Returns
    * ``x_curr`` The approximate root of the function.
    * ``iter_count`` The number of iterations taken.
    """
    x_prev = x0
    x_curr = x1
    iter_count = 0
    prev_x = None 
    
    while abs(f(x_curr)) > tol and iter_count < max_iter:
        x_next = x_curr - f(x_curr) * (x_curr - x_prev) / (f(x_curr) - f(x_prev))
        
        if prev_x is None or abs(x_curr - prev_x) > tol:
            print(f"Llamada i={iter_count+1}\t x={x_curr:.5f}\t y={f(x_curr):.2f}")
            prev_x = x_curr 
        x_prev = x_curr
        x_curr = x_next
        iter_count += 1

    return x_curr, iter_count



## EJEMPLO UNO

In [29]:
# Global counter
i = 0

# Define the function to be used
def func(x):
    global i
    i += 1
    y = x**3 - 3 * x**2 + x - 1
    return y

# Call the secant method
secant_method(func, x0=2, x1=3)

Llamada i=1	 x=3.00000	 y=2.00
Llamada i=2	 x=2.60000	 y=-1.10
Llamada i=3	 x=2.74227	 y=-0.20
Llamada i=4	 x=2.77296	 y=0.03
Llamada i=5	 x=2.76922	 y=-0.00
Llamada i=6	 x=2.76929	 y=-0.00


(2.7692923542484045, 6)

## EJERCICIO DOS

In [32]:
i = 0

def func(x):
    global i
    i += 1
    y = math.sin(x) + 0.5
    return y

secant_method(func, x0=2, x1=3)

Llamada i=1	 x=3.00000	 y=0.64
Llamada i=2	 x=3.83460	 y=-0.14
Llamada i=3	 x=3.68602	 y=-0.02
Llamada i=4	 x=3.66399	 y=0.00
Llamada i=5	 x=3.66520	 y=-0.00


(3.66519143172732, 5)

## SPLINES CUBICOS

In [ ]:
import sympy as sym
from IPython.display import display
from typing import List

def cubic_spline(xs: List[float], ys: List[float]) -> List[sym.Expr]:
    if len(xs) != len(ys):
        raise ValueError("Las listas xs e ys deben tener la misma longitud.")
    
    # Ordenar puntos por sus coordenadas x
    points = sorted(zip(xs, ys), key=lambda x: x[0])  
    xs = [x for x, _ in points]
    ys = [y for _, y in points]
    
    n = len(points) - 1  # Número de intervalos
    h = [xs[i + 1] - xs[i] for i in range(n)]  # Distancias entre puntos consecutivos
    
    # Cálculo de los coeficientes alpha
    alpha = [0] * (n + 1)
    for i in range(1, n):
        alpha[i] = (3 / h[i]) * (ys[i + 1] - ys[i]) - (3 / h[i - 1]) * (ys[i] - ys[i - 1])
    
    # Inicialización de las listas
    l = [1] + [0] * n
    u = [0] * n
    z = [0] * (n + 1)
    
    # Cálculo de los valores l, u y z
    for i in range(1, n):
        l[i] = 2 * (xs[i + 1] - xs[i - 1]) - h[i - 1] * u[i - 1]
        u[i] = h[i] / l[i]
        z[i] = (alpha[i] - h[i - 1] * z[i - 1]) / l[i]
    
    l[n] = 1  # Condición del spline natural
    z[n] = 0  # Condición del spline natural
    c = [0] * (n + 1)
    b = [0] * n
    d = [0] * n
    a = [ys[i] for i in range(n)]
    
    # Cálculo de los coeficientes del spline
    for j in range(n - 1, -1, -1):
        c[j] = z[j] - u[j] * c[j + 1]
        b[j] = ((ys[j + 1] - ys[j]) / h[j]) - h[j] * (c[j + 1] + 2 * c[j]) / 3
        d[j] = (c[j + 1] - c[j]) / (3 * h[j])
    
    # Construcción de los splines
    x = sym.Symbol("x")
    splines = []
    for j in range(n):
        S = a[j] + b[j] * (x - xs[j]) + c[j] * (x - xs[j])**2 + d[j] * (x - xs[j])**3
        splines.append(S)
    
    return splines


In [36]:
xs = [0, -1, 1]
ys = [-1, 1 , 3]

splines = cubic_spline(xs=xs, ys=ys)
print("Splines cubicos sin expandir")
_ = [display(s) for s in splines]
print("Splines cubicos expandidos")
_ = [display(s.expand()) for s in splines]

Splines cubicos sin expandir


-3.5*x + 1.5*(x + 1)**3 - 2.5

-1.5*x**3 + 4.5*x**2 + 1.0*x - 1

Splines cubicos expandidos


1.5*x**3 + 4.5*x**2 + 1.0*x - 1.0

-1.5*x**3 + 4.5*x**2 + 1.0*x - 1